Estimating the value of a used car is one of the main everyday challenges in the automotive business. We believe that the sales price of a car is not only based on the value of the product itself, but is also heavily influenced by things like market trends, current availability, and politics.
With this challenge, we hope to raise some interest in this exciting topic and also gain some insight into what the main factors are that drive the value of a used car.

The data provided consists of almost 5000 real BMW cars that were sold via a b2b auction in 2018. The price shown in the table is the highest bid that was reached during the auction.

We have already done some data cleanup and filtered out cars with engine damage etc. However there may still be minor damages like scratches, but we do not have more information about that.

We have also extracted 8 criteria based on the equipment of cars that we think might have a good impact on the value of a used car. These criteria have been labeled feature1 to feature8 and are shown in the data below.

## 0. Importing Libraries

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import bisect

import warnings
warnings.filterwarnings('ignore')

##  1. Reading data

In [7]:
df = pd.read_csv("./data/bmw_pricing_challenge.csv")

In [1]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.shape

In [ ]:
df.describe()

## 2. Exploratory Data Analysis

### 2.1. Missing Values

Check if the dataset contains any missing values.

### 2.2. Distribution of the target variable

Plot the probabilistic distribution of the target variable.

### 2.3. Distribution of numerical variables

Plot the probabilistic distribution of the numerical features.

### 2.4. Histogram of categorical variables

Plot the histogram of the categorical features.

## 3. Data Splitting

Split the dataset into 2 training and test datasets.

## 4. Feature Engineering

### 4.1. Removing non-predictive features

Remove any unnecessary feature.

### 4.2. Creating new features

Creating polynomial features for numerical columns

### 4.3. Scaling numerical variables

### 4.4. Categorical variables encoding

Convert categorical columns into numerical columns using label encoding or one-hot encoding.

### 4.5. Converting boolean columns

## 5. Linear Regression

Fit a linear regression model on the training set. Evaluate the model on the testing set. Use the $R^2$ as an evaluation metric.

Plot feature importance/weight.

## 6. Ridge

Fit a ridge regression model on the training set. Use cross-validation in order to tune the regularization parameter of the ridge model. Evaluate the model on the testing set. Use the $R^2$ as an evaluation metric.

Plot feature importance/weight.

## 7. Lasso

Fit a lasso regression model on the training set. Use cross-validation in order to tune the regularization parameter of the ridge model. Evaluate the model on the testing set. Use the $R^2$ as an evaluation metric.

Plot feature importance/weight.